In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# Clean the data

In [2]:
train = pd.read_csv('data/train.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train['Male'] = train['Sex'] == 'male'

In [5]:
train['Female'] = train['Sex'] == 'female'

In [6]:
train['Cherbourg'] = train['Embarked'] == 'C'
train['Queestown'] = train['Embarked'] == 'Q'
train['Southampton'] = train['Embarked'] == 'S'

In [7]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Male', 'Female',
       'Cherbourg', 'Queestown', 'Southampton'],
      dtype='object')

In [8]:
train[train.duplicated()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Male,Female,Cherbourg,Queestown,Southampton


In [39]:
cleaned_train = train[['Pclass', 'Age', 'SibSp',
       'Parch', 'Fare', 'Male', 'Female',
       'Cherbourg', 'Queestown', 'Southampton','Survived']]

In [40]:
cleaned_train.describe()

,Pclass,Age,SibSp,Parch,Fare,Survived
count,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208,0.383838
std,0.836071,14.526497,1.102743,0.806057,49.693429,0.486592
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [41]:
# clean age
cleaned_train['imputed_age'] = cleaned_train['Age'].isna()

<ipython-input-41-2f10c6a17c63>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_train['imputed_age'] = cleaned_train['Age'].isna()


In [42]:
cleaned_train['Age'].fillna(30,inplace=True)

<ipython-input-42-c280b058c77a>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_train['Age'].fillna(30,inplace=True)
<ipython-input-42-c280b058c77a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_train['Age'].fillna(30,inplace=True)


In [43]:
# normalize age
scaler = MinMaxScaler()
cleaned_train['normalized_age'] = scaler.fit_transform(cleaned_train[['Age']])


<ipython-input-43-b04f38f44805>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_train['normalized_age'] = scaler.fit_transform(cleaned_train[['Age']])


In [44]:
# normalize fare
# fare is very skewed to the right. use log
cleaned_train['normalized_fare'] = np.log1p(cleaned_train['Fare'])

<ipython-input-44-21dc89576571>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_train['normalized_fare'] = np.log1p(cleaned_train['Fare'])


In [45]:
cleaned_train.describe()

,Pclass,Age,SibSp,Parch,Fare,Survived,normalized_age,normalized_fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.758889,0.523008,0.381594,32.204208,0.383838,0.368672,2.962246
std,0.836071,13.002570,1.102743,0.806057,49.693429,0.486592,0.163390,0.969048
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.271174,2.187218
50%,3.000000,30.000000,0.000000,0.000000,14.454200,0.000000,0.371701,2.737881
75%,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,0.434531,3.465736
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,6.240917


In [26]:
# cleaned_train = cleaned_train[-cleaned_train['imputed_age']]


In [46]:
X = cleaned_train[['Pclass', 'normalized_age', 'SibSp',
       'Parch', 'normalized_fare', 'Male', 'Female',
       'Cherbourg', 'Queestown', 'Southampton'
#                    ,'imputed_age'
                  ]]

In [47]:
y = cleaned_train['Survived']

# Training

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8)

In [49]:
model = MLPClassifier(learning_rate_init=0.05,activation='relu'
                      ,hidden_layer_sizes=(256,128,64)
                     )

In [52]:
model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(256, 128, 64), learning_rate_init=0.05)

In [53]:
y_probs = model.predict_proba(X_test)[:, 1]

threshold = 0.5
y_pred_custom = (y_probs >= threshold).astype(int)
print(classification_report(y_test, y_pred_custom))

auc = roc_auc_score(y_test, y_probs)
print(f"AUC: {auc:.2f}")

              precision    recall  f1-score   support

           0       0.86      0.83      0.85       108
           1       0.76      0.79      0.77        71

    accuracy                           0.82       179
   macro avg       0.81      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179

AUC: 0.90


In [54]:
y_probs = model.predict_proba(X_test)[:, 1]

threshold = 0.4
y_pred_custom = (y_probs >= threshold).astype(int)
print(classification_report(y_test, y_pred_custom))

auc = roc_auc_score(y_test, y_probs)
print(f"AUC: {auc:.2f}")

              precision    recall  f1-score   support

           0       0.86      0.79      0.82       108
           1       0.71      0.80      0.75        71

    accuracy                           0.79       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.79      0.79       179

AUC: 0.90


In [55]:
y_probs = model.predict_proba(X_test)[:, 1]

threshold = 0.3
y_pred_custom = (y_probs >= threshold).astype(int)
print(classification_report(y_test, y_pred_custom))

auc = roc_auc_score(y_test, y_probs)
print(f"AUC: {auc:.2f}")

              precision    recall  f1-score   support

           0       0.87      0.74      0.80       108
           1       0.68      0.83      0.75        71

    accuracy                           0.78       179
   macro avg       0.77      0.79      0.77       179
weighted avg       0.79      0.78      0.78       179

AUC: 0.90


# Final prediction

In [56]:
final_test = pd.read_csv('data/test.csv')

In [57]:
final_test['Male'] = final_test['Sex'] == 'male'

final_test['Female'] = final_test['Sex'] == 'female'

final_test['Cherbourg'] = final_test['Embarked'] == 'C'
final_test['Queestown'] = final_test['Embarked'] == 'Q'
final_test['Southampton'] = final_test['Embarked'] == 'S'

In [77]:
final_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [58]:
cleaned_final_test = final_test[['Pclass', 'Age', 'SibSp',
       'Parch', 'Fare', 'Male', 'Female',
       'Cherbourg', 'Queestown', 'Southampton']]

In [78]:
cleaned_final_test = final_test[['Pclass', 'Age', 'SibSp',
       'Parch', 'Fare', 'Male', 'Female',
       'Cherbourg', 'Queestown', 'Southampton']]

cleaned_final_test.describe()

# clean age
# cleaned_final_test['imputed_age'] = cleaned_final_test['Age'].isna()

cleaned_final_test['Age'].fillna(30,inplace=True)
cleaned_final_test['Fare'].fillna(35.627,inplace=True)

# normalize age
scaler = MinMaxScaler()
cleaned_final_test['normalized_age'] = scaler.fit_transform(cleaned_final_test[['Age']])


# normalize fare
# fare is very skewed to the right. use log
cleaned_final_test['normalized_fare'] = np.log1p(cleaned_final_test['Fare'])

cleaned_final_test = cleaned_final_test[['Pclass', 'normalized_age', 'SibSp',
       'Parch', 'normalized_fare', 'Male', 'Female',
       'Cherbourg', 'Queestown', 'Southampton'
#                    ,'imputed_age'
                  ]]

<ipython-input-78-76044ad3f17f>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_final_test['Age'].fillna(30,inplace=True)
<ipython-input-78-76044ad3f17f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_final_test['Age'].fillna(30,inplace=True)
<ipython-input-78-76044ad3f17f>:11: FutureWarning: A value is trying to be set on

In [79]:
y_probs = model.predict_proba(cleaned_final_test)[:, 1]

threshold = 0.5
y_pred_custom = (y_probs >= threshold).astype(int)

In [80]:
y_pred_custom

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [81]:
final_test['Survived'] = y_pred_custom

In [82]:
final_test[['PassengerId','Survived']].to_csv('output.csv')